## Question 2

In [3]:
# Importation des librairies de base

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from scipy.stats import gaussian_kde    
from scipy.stats import probplot
from scipy.stats import ttest_rel

import wrds
from scipy import stats
from scipy.stats import skew, kurtosis
import warnings

from sklearn.linear_model import RidgeCV, LassoCV, ElasticNetCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.stattools import adfuller
import warnings
warnings.filterwarnings('ignore')

## Question 2

### Importation des Données de la Question 1

Pour la Question 2, nous utilisons les **mêmes données que la Question 1** :
- **Variable dépendante** : Rendement excédentaire du secteur `Finan` (Services financiers)
- **Régresseurs** : Les 6 facteurs de Fama-French (Mkt-RF, SMB, HML, RMW, CMA, Mom)
- **Période** : Janvier 1980 - Décembre 2021 (504 observations mensuelles)

Les données proviennent de :
1. Ken French Data Library - 17 Industry Portfolios (Value-Weighted Returns)
2. Ken French Data Library - Fama-French 5 Factors
3. Ken French Data Library - Momentum Factor

In [4]:
# Chargement des données depuis les fichiers CSV (même processus que Q1)

# 1. Charger les 17 Industry Portfolios
df_17_vw = pd.read_csv('17_Industry_Portfolios.csv', skiprows=11, nrows=1191)
df_17_vw.columns = df_17_vw.columns.str.strip()
df_17_vw = df_17_vw.rename(columns={df_17_vw.columns[0]: 'Date'})
df_17_vw['Date'] = df_17_vw['Date'].astype(str).str.strip()
df_17_vw = df_17_vw[df_17_vw['Date'].str.len() == 6]
df_17_vw = df_17_vw[(df_17_vw['Date'] >= '198001') & (df_17_vw['Date'] <= '202112')].copy()
for col in df_17_vw.columns[1:]:
    df_17_vw[col] = pd.to_numeric(df_17_vw[col], errors='coerce')

# 2. Charger les 5 facteurs Fama-French
df_FF5 = pd.read_csv('F-F_Research_Data_5_Factors_2x3.csv', skiprows=3)
df_FF5.columns = df_FF5.columns.str.strip()
df_FF5 = df_FF5.rename(columns={df_FF5.columns[0]: 'Date'})
df_FF5['Date'] = df_FF5['Date'].astype(str).str.strip()
df_FF5 = df_FF5[df_FF5['Date'].str.len() == 6]
df_FF5 = df_FF5[(df_FF5['Date'] >= '198001') & (df_FF5['Date'] <= '202112')].copy()
for col in df_FF5.columns[1:]:
    df_FF5[col] = pd.to_numeric(df_FF5[col], errors='coerce')

# 3. Charger le facteur Momentum
df_Mom = pd.read_csv('F-F_Momentum_Factor.csv', skiprows=13)
df_Mom.columns = df_Mom.columns.str.strip()
df_Mom = df_Mom.rename(columns={df_Mom.columns[0]: 'Date'})
df_Mom['Date'] = df_Mom['Date'].astype(str).str.strip()
df_Mom = df_Mom[df_Mom['Date'].str.len() == 6]
df_Mom = df_Mom[(df_Mom['Date'] >= '198001') & (df_Mom['Date'] <= '202112')].copy()
for col in df_Mom.columns[1:]:
    df_Mom[col] = pd.to_numeric(df_Mom[col], errors='coerce')

# 4. Fusionner les datasets
df_6factors = pd.merge(df_FF5, df_Mom, on='Date', how='inner')
df_final = pd.merge(df_6factors, df_17_vw, on='Date', how='inner')

# 5. Créer les rendements excédentaires pour tous les secteurs
facteur_cols = ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF', 'Mom']
secteur_cols = [col for col in df_final.columns if col not in ['Date'] + facteur_cols]

# Créer le DataFrame final avec rendements excédentaires
cols_to_keep = ['Date', 'Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'Mom', 'RF']
df_final_excess = df_final[cols_to_keep].copy()

for secteur in secteur_cols:
    df_final_excess[secteur] = df_final[secteur] - df_final['RF']

print(f"✓ Données chargées : {df_final_excess.shape[0]} observations × {df_final_excess.shape[1]} colonnes")
print(f"✓ Période : {df_final_excess['Date'].iloc[0]} à {df_final_excess['Date'].iloc[-1]}")
print(f"✓ Secteurs disponibles : {secteur_cols}")

df_final_excess

✓ Données chargées : 504 observations × 25 colonnes
✓ Période : 198001 à 202112
✓ Secteurs disponibles : ['Food', 'Mines', 'Oil', 'Clths', 'Durbl', 'Chems', 'Cnsum', 'Cnstr', 'Steel', 'FabPr', 'Machn', 'Cars', 'Trans', 'Utils', 'Rtail', 'Finan', 'Other']


,Date,Mkt-RF,SMB,HML,RMW,CMA,Mom,RF,Food,Mines,...,Cnstr,Steel,FabPr,Machn,Cars,Trans,Utils,Rtail,Finan,Other
0,198001,5.50,1.88,1.85,-1.84,1.89,7.45,0.80,3.06,13.29,...,7.81,16.78,6.42,6.94,9.96,15.07,0.75,-0.63,1.31,3.10
1,198002,-1.23,-1.62,0.59,-0.95,2.92,7.89,0.89,-6.18,2.89,...,-3.01,-0.60,-1.91,-4.74,-7.42,-7.37,-4.31,-6.26,-5.91,-2.26
2,198003,-12.89,-6.97,-0.96,1.82,-1.05,-9.58,1.21,-10.24,-22.35,...,-16.08,-19.70,-17.92,-13.98,-10.77,-14.15,-7.81,-9.26,-10.13,-11.02
3,198004,3.96,1.05,1.03,-2.18,0.34,-0.48,1.26,5.79,1.10,...,2.77,1.04,2.69,-1.16,-4.15,-1.88,10.78,3.50,5.62,5.26
4,198005,5.26,2.00,0.38,0.43,-0.63,-1.18,0.81,7.16,9.18,...,8.06,4.39,4.89,4.07,2.32,5.06,3.79,9.45,7.06,5.52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,202108,2.95,-0.66,-0.23,-0.26,-1.72,2.63,0.00,-0.41,-2.59,...,1.82,3.51,5.29,3.52,1.98,-0.67,3.23,3.02,2.86,3.93
500,202109,-4.40,1.12,5.12,-1.92,2.00,1.39,0.00,-3.39,-8.11,...,-3.71,-10.64,-5.68,-5.72,4.04,-2.11,-4.87,-5.43,-1.54,-5.74
501,202110,6.63,-2.70,-0.53,1.71,-1.54,3.32,0.00,3.42,8.68,...,11.42,10.14,3.71,7.57,30.82,6.06,5.11,3.77,7.07,5.51
502,202111,-1.58,-1.70,-0.37,7.19,1.70,0.96,0.00,-2.92,-0.18,...,5.31,-6.28,0.25,6.76,2.70,-5.46,-1.97,1.16,-5.68,-2.84


In [5]:
# Préparation des données pour la Question 2
# Choix du secteur : Finan (Services financiers)
# Justification : Secteur financier connu pour ses queues épaisses (crises, volatilité)

secteur_choisi = 'Finan'  

# Variable dépendante : rendement excédentaire du secteur
y = df_final_excess[secteur_choisi].values

# Régresseurs : les 6 facteurs de Fama-French
facteurs = ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'Mom']
X = df_final_excess[facteurs].values

# Ajouter une constante pour la régression
X_with_const = sm.add_constant(X)

print("="*80)
print("PRÉPARATION DES DONNÉES POUR LA QUESTION 2")
print("="*80)
print(f"\nSecteur choisi : {secteur_choisi} (Services financiers)")
print(f"Nombre d'observations : {len(y)}")
print(f"Nombre de régresseurs : {len(facteurs)} facteurs + constante = {X_with_const.shape[1]}")

print(f"\n{'Statistiques descriptives - Variable dépendante:':<60}")
print(f"  Moyenne : {np.mean(y):.4f}%")
print(f"  Écart-type : {np.std(y):.4f}%")
print(f"  Minimum : {np.min(y):.4f}%")
print(f"  Maximum : {np.max(y):.4f}%")
print(f"  Asymétrie (skewness) : {skew(y):.4f}")
print(f"  Aplatissement (kurtosis) : {kurtosis(y):.4f}")

if kurtosis(y) > 3:
    print(f"\n  → Kurtosis > 3 : Distribution à QUEUES ÉPAISSES détectée !")
    print(f"  → Le modèle Student-t sera probablement plus approprié que le modèle Normal")
else:
    print(f"\n  → Kurtosis ≈ 3 : Distribution proche de la normale")

print("\n" + "="*80)

PRÉPARATION DES DONNÉES POUR LA QUESTION 2

Secteur choisi : Finan (Services financiers)
Nombre d'observations : 504
Nombre de régresseurs : 6 facteurs + constante = 7

Statistiques descriptives - Variable dépendante:            
  Moyenne : 0.7948%
  Écart-type : 5.4985%
  Minimum : -22.4000%
  Maximum : 17.1200%
  Asymétrie (skewness) : -0.6141
  Aplatissement (kurtosis) : 2.2063

  → Kurtosis ≈ 3 : Distribution proche de la normale

